<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/train_llama2_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# llama2.c

In [ ]:
!git clone https://github.com/karpathy/llama2.c

In [2]:
%cd llama2.c

/content/llama2.c


In [ ]:
!pip install -q -r requirements.txt

In [5]:
!python tinystories.py download

data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [01:20<00:00, 19.9MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [ ]:
!python tinystories.py pretokenize


In [7]:
!ls -hlg data/TinyStories_all_data

total 8.5G
-rw-r--r-- 1 root       40M Dec 22 02:03 data00.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data00.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data01.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data01.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data02.bin
-rw-r--r-- 1 300000513 135M May 17  2023 data02.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data03.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data03.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data04.bin
-rw-r--r-- 1 300000513 135M May 17  2023 data04.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data05.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data05.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data06.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data06.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data07.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data07.json
-rw-r--r-- 1 root       40M Dec 22 02:04 data08.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data08.json
-rw-r--r-- 1 root       40M Dec 22 02:04 dat

In [ ]:
#!python -m train.py --compile=False --eval_iters=10 --batch_size=80 --dtype=float16
# On T4-16G just example, eval_iters=1 1 eval_iters need 16h or more
!python -m train.py --compile=False --eval_iters=1 --batch_size=90 --dtype=float16

流式输出内容被截断，只能显示最后 5000 行内容。
16173 | loss 1.2484 | lr 4.715766e-04 | 1374.51ms | mfu 2.07%
16174 | loss 1.1746 | lr 4.715729e-04 | 1365.96ms | mfu 2.07%
16175 | loss 1.2539 | lr 4.715693e-04 | 1375.74ms | mfu 2.07%
16176 | loss 1.1725 | lr 4.715656e-04 | 1372.87ms | mfu 2.07%
16177 | loss 1.2576 | lr 4.715619e-04 | 1366.48ms | mfu 2.08%
16178 | loss 1.2322 | lr 4.715582e-04 | 1371.58ms | mfu 2.08%
16179 | loss 1.2033 | lr 4.715546e-04 | 1367.77ms | mfu 2.08%
16180 | loss 1.2241 | lr 4.715509e-04 | 1373.55ms | mfu 2.08%
16181 | loss 1.1826 | lr 4.715472e-04 | 1374.92ms | mfu 2.08%
16182 | loss 1.2620 | lr 4.715435e-04 | 1375.73ms | mfu 2.08%
16183 | loss 1.2309 | lr 4.715399e-04 | 1368.96ms | mfu 2.08%
16184 | loss 1.2852 | lr 4.715362e-04 | 1373.42ms | mfu 2.08%
16185 | loss 1.2662 | lr 4.715325e-04 | 1369.09ms | mfu 2.08%
16186 | loss 1.2001 | lr 4.715288e-04 | 1368.15ms | mfu 2.08%
16187 | loss 1.1977 | lr 4.715252e-04 | 1372.44ms | mfu 2.08%
16188 | loss 1.2633 | lr 4.715215e-04 | 136

GPU 显存打满
```
nvidia-smi --query-gpu=timestamp,memory.total,memory.free,memory.used,name,utilization.gpu,utilization.memory --format=csv -l 3
timestamp, memory.total [MiB], memory.free [MiB], memory.used [MiB], name, utilization.gpu [%], utilization.memory [%]
2023/12/22 02:48:14.470, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 97 %
2023/12/22 02:48:17.472, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 99 %
2023/12/22 02:48:20.473, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 97 %
2023/12/22 02:48:23.474, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 100 %
2023/12/22 02:48:26.477, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 97 %
2023/12/22 02:48:29.479, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 95 %
2023/12/22 02:48:32.481, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 97 %
```

# llama2-zh.c


In [1]:
%cd /content
!git clone https://github.com/chenyangMl/llama2.c-zh.git
%cd llama2.c-zh

/content
Cloning into 'llama2.c-zh'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 122 (delta 57), reused 94 (delta 31), pack-reused 0
Receiving objects: 100% (122/122), 1.96 MiB | 1.91 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/llama2.c-zh


In [ ]:
!pip install -r requirements.txt

In [5]:
# 下载中英文数据
!python dataProcess/tinystories.py download


data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [00:16<00:00, 100MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is g

In [ ]:
# tokenizer 数据
!python dataProcess/tinystories.py pretokenize

In [ ]:
!python -m train.py --compile=False --eval_iters=10 --batch_size=8 --dtype=float16

## 小说数据集训练
- https://github.com/shjwudp/shu/tree/master/books


In [ ]:
!python dataProcess/tinyfictions.py download

In [ ]:
!python dataProcess/tinyfictions.py pretokenize


In [ ]:
# 训练. 修改batch_size=256, vocab_size=自定义的词表长度, 其他根据需要进行调整
#!python train.py
# 单卡
!python -m train.py --compile=False --eval_iters=10 --batch_size=8


# baby-llama2-chinese

In [ ]:
%cd /content
!git clone https://github.com/DLLXW/baby-llama2-chinese.git

/content
Cloning into 'baby-llama2-chinese'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 73 (delta 11), reused 11 (delta 11), pack-reused 47
Receiving objects: 100% (73/73), 479.85 KiB | 19.19 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
%cd /content/baby-llama2-chinese

/content/baby-llama2-chinese


# 更多中文数据集和模型参考

- https://github.com/crownpku/awesome-chinese-nlp#corpus-%E4%B8%AD%E6%96%87%E8%AF%AD%E6%96%99
- https://github.com/brightmart/nlp_chinese_corpus
- NLP民工的乐园(百科全书): **https://github.com/fighting41love/funNLP**
- 大模型汇总： https://github.com/chenking2020/FindTheChatGPTer
- 超赞的超大规模中文语料集： https://github.com/esbatmop/MNBVC